<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/BERT_d5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# نصب کتابخانه‌های موردنیاز
!pip install --upgrade pip setuptools wheel
!pip install tensorflow transformers scikit-learn pandas numpy

import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# بارگذاری دیتاست از گوگل کولب (لینک فایل را جایگزین کنید)
df = pd.read_csv("d5.csv")  # جایگزین کنید با مسیر صحیح فایل در کولب

# ستون‌های ورودی و لیبل را مشخص کنید
X = df.drop(columns=['F']).astype(str)  # ویژگی‌ها را به فرمت متنی تبدیل می‌کنیم
y = df['F'].values  # لیبل‌ها

# تبدیل ویژگی‌ها به یک لیست از رشته‌ها
X = X.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist()

# تقسیم دیتاست به مجموعه آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# بارگذاری tokenizer برای BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    return tokenizer(
        data,
        padding=True,
        truncation=True,
        return_tensors='tf',
        max_length=50  # تنظیم مقدار مناسب بر اساس طول داده‌ها
    )

# تبدیل داده‌ها به فرمت BERT
train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)

# تعریف مدل BERT برای دسته‌بندی دودویی
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# کامپایل مدل
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# آموزش مدل
history = model.fit(
    x={
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    },
    y=y_train,
    validation_split=0.1,
    batch_size=16,
    epochs=3
)

# ارزیابی مدل روی داده‌های تست
y_pred_logits = model.predict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

y_pred = np.argmax(y_pred_logits.logits, axis=1)

# محاسبه دقت و F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
450/450 [==============================] - 1328s 3s/step - loss: 0.3082 - accuracy: 0.9089 - val_loss: 0.2683 - val_accuracy: 0.9250
Epoch 2/3
450/450 [==============================] - 1289s 3s/step - loss: 0.3092 - accuracy: 0.9089 - val_loss: 0.2663 - val_accuracy: 0.9250
Epoch 3/3
63/63 [==============================] - 59s 856ms/step
Accuracy: 0.9105
F1-Score: 0.9532
